In [1]:
# Ping Pong game with sounds and music

import pygame, sys 


# Sprite is a simple base class for visible game objects
# use Sprite base class from sprite module from pygame module
class Ball(pygame.sprite.Sprite): 
    
    # constructor of Ball class
    def __init__(self, image_file, speed, location = [0,0]): 
        
        # instantiate base class
        pygame.sprite.Sprite.__init__(self)  
        
        # assign values to Ball object attributes

        self.image = pygame.image.load(image_file)   # load image of Ball object (ball)      
        self.rect = self.image.get_rect()            # get rectangle surrounding the ball image
        self.speed = speed                           # set ball speed
        self.rect.left, self.rect.top = location     # set ball initial location (top, left)
         
 
    def move(self): 
        global points, score_text 
        
        self.rect = self.rect.move(self.speed)    # move the ball rectangle with its speed
       
        # if ball rectangle hits the left or right edge of screen
        if self.rect.left < 0 or self.rect.right > screen.get_width(): 
            self.speed[0] = -self.speed[0]        # reverse ball direction
            
            
            # if top of ball rectangle still within the screen, means less than screen hight
            if self.rect.top < screen.get_height():  
                hit_wall.play()     # play sound
                
                                               
        if self.rect.top <= 0 : # if ball rectangle hits the top edge of screen
            
            self.speed[1] = -self.speed[1] # revers ball direction
            points = points + 1            # player gets a point
            
            score_text = font.render(str(points), 1, (0, 0, 255)) # update text_score with new score
            get_point.play()  # play sound when the ball hits the top (player gets a point)
            

# another subclass of Sprite class
class Paddle(pygame.sprite.Sprite):
    
    # constructor of Paddle
    def __init__(self, location = [0,0]): 
        
        # instantiate base class
        pygame.sprite.Sprite.__init__(self) 
        
        # assign values to Paddle object attributes
       
        image_surface = pygame.Surface([100, 20])   # create a paddle surface 
        image_surface.fill([255,0,0])               # fill paddle with red
        self.image    = image_surface.convert()    # change the pixel format of an image
        self.rect = self.image.get_rect()           # get rectangle surrounding the ball image
        self.rect.left, self.rect.top = location    # set initial location of the ball (top, left)


# MAIN CODE
       
pygame.init()         # initialize all imported pygame modules
pygame.mixer.init()   # initialize mixer module for Sound loading and playback


pygame.mixer.music.load("bg_music.mp3")          # Load background music
pygame.mixer.music.set_volume(0.3)               # Set volume for music
pygame.mixer.music.play(-1)                      # Play music forever

# ---------------------------------------- #
# Load other sounds and set their volumes
hit = pygame.mixer.Sound("hit_paddle.wav")          
hit.set_volume(0.4)                                    
new_life = pygame.mixer.Sound("new_life.wav")       
new_life.set_volume(0.5)                            
splat = pygame.mixer.Sound("splat.wav")             
splat.set_volume(0.6)                               
hit_wall = pygame.mixer.Sound("hit_wall.wav")       
hit_wall.set_volume(0.4)                            
get_point = pygame.mixer.Sound("get_point.wav")     
get_point.set_volume(0.2)                           
bye = pygame.mixer.Sound("game_over.wav")           
bye.set_volume(0.6)   
# ---------------------------------------- #



screen = pygame.display.set_mode([640,480])   # initialize a window or screen for display
clock = pygame.time.Clock()                   # create an object to help track time

myBall = Ball('pong.png', [15, 10], [50, 50])  # create a new Ball object

ballGroup = pygame.sprite.Group(myBall)       # add myBall to group of sprites
                                              # the player can collide with without dying
    
paddle = Paddle([270, 400])  # create a Paddle object
lives = 5                    # initialize number of lives
points = 0                   # initialize player points

# create a new Font object from a file
# Font(filename, size), size is the height of the font in pixels
font = pygame.font.Font(None, 50) 

# draw text on a new Surface
# render(text, antialias, color), text is only single line
# antialias is boolean: if 1 (true) the characters will have smooth edges
score_text = font.render(str(points), 1, (0, 0, 255)) 

textpos = [10, 10]   # position of text on main screen (top,left)
done = False         # game is not done

# main game loop
running = True 
while running: 
    
    # the program runs at most 30 frames per second
    clock.tick(30) 
    screen.fill([255,255,255])          # fill screen color
    
    for event in pygame.event.get():    # get events from the queue
        if event.type == pygame.QUIT:   # if the Event object’s type is equal to the constant QUIT
            running = False             # break the main loop to stop the game
        elif event.type == pygame.MOUSEMOTION:  # if the Event object’s type is equal to the constant MOUSEMOTION
            paddle.rect.centerx = event.pos[0] 
            
    # return a list containing all Sprites in a Group that intersect with another Sprite
    # spritecollide(sprite, group, dokill)
    # dokill is boolean, if 1 (true), all Sprites that collide will be removed from the Group
    if pygame.sprite.spritecollide(paddle, ballGroup, False): 
        hit.play()                              # Play sound when the ball hits the paddle
        myBall.speed[1] = -myBall.speed[1]      # reverse the ball direction
 
    # move the ball
    myBall.move() 
 
    # if the game still running
    if not done:
        # blit(source, dest)
        # draw source image onto dest surface
        screen.blit(myBall.image, myBall.rect) 
        screen.blit(paddle.image, paddle.rect) 
        
        # dest can be pair of coordinates representing the upper left corner of the source
        screen.blit(score_text, textpos) 
        
        width = screen.get_width() 
        
        # for each of the 5 balls (lives)
        for i in range (lives): 
            screen.blit(myBall.image, [width - 40 * i, 430]) 
            
        # update the contents of the entire display    
        pygame.display.flip() 
 
    if myBall.rect.top >= screen.get_rect().bottom: 
        if not done: 
            splat.play()           # play sound when the player loses a life
        lives = lives - 1 
        if lives <= 0: 
            #if not done:
            pygame.time.delay(1000)             # pause the program for 1 second  
            bye.play()                          # then play the ending sound
           
            # prepare GAME OVER message with player score
            final_text1 = "Game Over" 
            final_text2 = "Your final score is:  " + str(points) 
            
            # create Font and Surface objects for text
            ft1_font = pygame.font.Font(None, 70) 
            ft1_surf = font.render(final_text1, 1, (0, 150, 150)) 
            
            ft2_font = pygame.font.Font(None, 50) 
            ft2_surf = font.render(final_text2, 1, (0, 150, 150)) 
            
            screen.blit(ft1_surf, [screen.get_width()/2 - ft1_surf.get_width()/2, 100]) 
            screen.blit(ft2_surf, [screen.get_width()/2 - ft2_surf.get_width()/2, 200]) 

            # update the contents of the entire display
            pygame.display.flip() 
  
            done = True                             # game is over
            pygame.mixer.music.fadeout(2000)        # fade out the music
        
        else: 
            pygame.time.delay(1000)                 # pause the program for 1 second         
            new_life.play()                         # play sound when a new life starts
            myBall.rect.topleft = [50, 50]          # set top left corner of myBall rect
            screen.blit(myBall.image, myBall.rect)  # draw ball on rect
            pygame.display.flip() 

            pygame.time.delay(1000)                 # pause the program for 1 second

# quit pygame to uninitialize all pygame modules that have previously been initialized     
pygame.quit()


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
